In [15]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0
)

### Project 2-1. 콤마 구분 리스트 파서 활용

In [17]:
parser = CommaSeparatedListOutputParser()
prompt = ChatPromptTemplate.from_template(
    """관심 분야와 관련된 한국의 유명한 장소나 활동 5가지를 콤마로 구분해 한 줄로 추천해 주세요.
관심 분야: {interest}
#Format: {format_instructions}
"""
)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 체인 생성
chain = prompt | llm | parser

# 테스트 입력
user_input = "음식"
result = chain.invoke({"interest": user_input})
print(result)

['서울의 N서울타워', '이태원의 먹자골목', '광화문의 한정식 거리', '부산의 자갈치 시장', '대전의 성심당']


### Project 2-2. 영화 리뷰 감정 분석기

In [23]:
from enum import Enum
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import OutputParserException

# 1. 감정 Enum 정의
class Sentiment(str, Enum):
    긍정 = "긍정"
    부정 = "부정"
    중립 = "중립"

# 2. 출력 파서 준비
parser = EnumOutputParser(enum=Sentiment)
format_instructions = parser.get_format_instructions()

# 3. 프롬프트 템플릿 준비
prompt = ChatPromptTemplate.from_template(
    """
    당신은 텍스트 감정 분석 전문가입니다.
    다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

    텍스트: "{text}"

    {format_instructions}

    규칙:
    1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력
    2. 다른 설명, 부가 설명, 이모지, 특수문자 추가 금지
    3. 오직 한 단어만 출력
    """
)
prompt = prompt.partial(format_instructions=format_instructions)

# 4. OutputFixingParser로 robust하게 파싱
fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

# 5. 감정 분석 체인 생성
chain = prompt | llm | fixing_parser

# 6. 테스트용 텍스트 리스트
texts = [
    "이 제품 정말 좋아요! 완전 만족스러워요.",
    "서비스가 너무 느리고 불친절했습니다.",
    "오늘은 비가 온다네요.",
    "배송은 빠르지만 품질이 아쉽습니다.",
    "최고의 경험이었습니다!",
    "완전 실망했어요... 최악이에요"
]

# 7. 감정 분석 실행
for i, text in enumerate(texts, 1):
    try:
        result = chain.invoke({"text": text})
        print(f"{i}. 텍스트: {text}\n→ 감정: {result.value}\n")
    except OutputParserException as e:
        print(f"{i}. 텍스트: {text}\n→ 파싱 오류: {str(e)[:100]}...\n")
    except Exception as e:
        print(f"{i}. 텍스트: {text}\n→ 기타 오류: {str(e)[:100]}...\n")


1. 텍스트: 이 제품 정말 좋아요! 완전 만족스러워요.
→ 감정: 긍정

2. 텍스트: 서비스가 너무 느리고 불친절했습니다.
→ 감정: 부정

3. 텍스트: 오늘은 비가 온다네요.
→ 감정: 중립

4. 텍스트: 배송은 빠르지만 품질이 아쉽습니다.
→ 감정: 부정

5. 텍스트: 최고의 경험이었습니다!
→ 감정: 긍정

6. 텍스트: 완전 실망했어요... 최악이에요
→ 감정: 부정



### Project 2-3. 학생 정보 구조화 시스템

In [33]:
from pydantic import BaseModel, Field
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 1. Pydantic 모델 정의
class StudentInfo(BaseModel):
    name: str = Field(..., description="학생의 이름")
    age: int = Field(..., description="학생의 나이(숫자)")
    major: str = Field(..., description="전공")
    hobbies: List[str] = Field(..., description="취미 리스트 (문자열의 리스트)")
    goal: str = Field(..., description="학생의 목표")

# 2. 파서 준비
parser = PydanticOutputParser(pydantic_object=StudentInfo)
format_instructions = parser.get_format_instructions()

# 3. 프롬프트 템플릿 준비
prompt = ChatPromptTemplate.from_template(
    """
아래 자기소개 텍스트에서 학생의 이름(name), 나이(age), 전공(major), 취미(hobbies: 리스트), 목표(goal)를 각각 추출해 StudentInfo JSON 구조로 반환하세요.
설명, 코드, 마크다운, 해설, 문장 모두 금지! 반드시 JSON만 출력!

예시:
{{
  "name": "홍길동",
  "age": 21,
  "major": "수학",
  "hobbies": ["등산", "수영"],
  "goal": "수학자"
}}

자기소개: {text}

{format_instructions}
"""
)
prompt = prompt.partial(format_instructions=format_instructions)

# 4. 체인 연결
chain = prompt | llm | parser

# 5. 테스트 입력
input_text = (
    "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. "
    "취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
)

# 6. 실행 및 출력 (pydantic v2 기준)
result = chain.invoke({"text": input_text})

print(result)  # Pydantic 객체
print(result.model_dump_json(indent=2))


name='김민수' age=22 major='컴퓨터공학' hobbies=['게임하기', '영화보기', '코딩'] goal='훌륭한 개발자'
{
  "name": "김민수",
  "age": 22,
  "major": "컴퓨터공학",
  "hobbies": [
    "게임하기",
    "영화보기",
    "코딩"
  ],
  "goal": "훌륭한 개발자"
}


### Project 2-4. 여행 계획 분석기

In [35]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import ChatPromptTemplate

# 1. 5개 필드 정의 (ResponseSchema)
schemas = [
    ResponseSchema(name="destination", description="여행한(할) 도시나 지역명"),
    ResponseSchema(name="duration", description="여행 기간 (예: '2박 3일', '3일간')"),
    ResponseSchema(name="budget", description="여행 전체 예산 (예: '30만원', '100달러')"),
    ResponseSchema(name="rating", description="추천도 (1~5점 중 숫자만)"),
    ResponseSchema(name="activities", description="여행의 주요 활동 리스트 (최소 2개, 예: '해운대 바다구경', '시장 구경')")
]

parser = StructuredOutputParser.from_response_schemas(schemas)
format_instructions = parser.get_format_instructions()

# 2. 프롬프트 템플릿
prompt = ChatPromptTemplate.from_template(
    """
아래 여행 후기(또는 계획)에서 핵심 정보를 추출해 구조화된 JSON만 반환하세요.

{format_instructions}

여행 텍스트:
{text}
"""
)
prompt = prompt.partial(format_instructions=format_instructions)

# 3. 체인 연결
chain = prompt | llm | parser

# 4. 테스트 입력
input_text = (
    "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, "
    "자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."
)

# 5. 실행 및 출력
result = chain.invoke({"text": input_text})

from pprint import pprint
pprint(result)


{'activities': '해운대 바다구경, 자갈치시장 구경, 감천문화마을 구경',
 'budget': '30만원',
 'destination': '부산',
 'duration': '2박3일',
 'rating': '4'}
